# NBA 1990-1991 Season Analysis

# Imports

In [12]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



import psycopg2
from dotenv import load_dotenv

from scipy.stats import kurtosis, skew
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, RepeatedKFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer, r2_score,explained_variance_score


# Models

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor


# Constants

In [4]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/NBA Projects/1990-1991-Season-Analysis/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME = "salaries_and_stats"

In [9]:
conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)

query = f"SELECT * FROM {TABLE_NAME} where season = '1990-1991';"
df = pd.read_sql_query(query, conn)

conn.close()

/Users/maukanmir/miniforge3/envs/machine-learning-env/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
df

,salary,Name,Team,GP,MPG,PPG,FGM,FGA,FG%,3PM,...,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,season
0,2500000.0,Michael Jordan,CHI,82,37.0,31.5,12.1,22.4,0.539,0.4,...,8.2,0.851,1.4,4.6,6.0,5.5,2.7,1.0,2.5,1990-1991
1,2260000.0,Karl Malone,UTA,82,40.3,29.0,10.3,19.6,0.527,0.0,...,10.8,0.770,2.9,8.9,11.8,3.3,1.1,1.0,3.0,1990-1991
2,1600000.0,Bernard King,WAS,64,37.5,28.4,11.1,23.6,0.472,0.1,...,7.6,0.790,1.8,3.2,5.0,4.6,0.9,0.3,4.0,1990-1991
3,2900000.0,Charles Barkley,PHI,67,37.3,27.6,9.9,17.4,0.570,0.7,...,9.8,0.722,3.9,6.3,10.1,4.2,1.6,0.5,3.1,1990-1991
4,4250000.0,Patrick Ewing,NYK,81,38.3,26.6,10.4,20.3,0.514,0.0,...,7.7,0.745,2.4,8.8,11.2,3.0,1.0,3.2,3.6,1990-1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,550000.0,Larry Drew,LAL,48,10.3,2.9,1.1,2.6,0.432,0.3,...,0.5,0.773,0.1,0.6,0.7,2.5,0.3,0.0,1.0,1990-1991
239,575000.0,Elden Campbell,LAL,52,7.3,2.8,1.1,2.4,0.455,0.0,...,0.9,0.653,0.8,1.1,1.8,0.2,0.2,0.7,0.3,1990-1991
240,150000.0,Scott Williams,CHI,51,6.6,2.5,1.0,2.0,0.510,0.0,...,0.5,0.714,0.8,1.1,1.9,0.3,0.2,0.3,0.5,1990-1991
241,1100000.0,Wayne Cooper,POR,67,11.1,2.2,0.9,2.2,0.393,0.0,...,0.6,0.786,0.8,2.0,2.8,0.3,0.1,0.9,0.3,1990-1991


# Functions

In [13]:
def check_for_nulls_dupes(df):
  print(f"The amount of Null Values: {df.isna().sum()}")
  print(f"The amount of Duplicated Values {df.duplicated().sum()}")


# EDA Analysis

In [14]:
check_for_nulls_dupes(df)

The amount of Null Values: salary    0
Name      0
Team      0
GP        0
MPG       0
PPG       0
FGM       0
FGA       0
FG%       0
3PM       0
3PA       0
3P%       0
FTM       0
FTA       0
FT%       0
ORB       0
DRB       0
RPG       0
APG       0
SPG       0
BPG       0
TOV       0
season    0
dtype: int64
The amount of Duplicated Values 0
